# Описание

Этот ноутбук содержит реализацию метода Шепли-Сноу для некоторых задач из области игр поиска.

Метод Шепли-Сноу используется для нахождения оптимальных стратегий и значения игры в прямоугольных играх (антагонистических играх конечного размера). Поскольку рассматриваемые игры поиска как раз относятся к играм такого типа, данный метод подходит здесь идеально.

Сначала будет представлено описание постановленной задачи, затем будут представлены импорты и реализация необходимых для работы программы функций. После чего (см. заголовки) будут представлены несколько режимов работы с ноутбуком: ввести параметры и получить ответ; проверка совпадения ответов модифицированной и немодифицированной версий; вывод интерактивного графика решений; ввести параметры и получить ответ для обобщения рассматриваемой задачи.

# Постановка задачи

Задача, решаемая данным ноутбуком, может быть сформулирована следующим образом.

Пусть у нас есть два игрока — $H$ и $S$, один объект и $n$ ячеек, куда этот объект можно положить. Игрок $H$ прячет этот объект в одну из $n$ ячеек, а игрок $S$ пытается за $m$ попыток его найти. Вероятность найти объект в $i$-ой ячейке, если он там содержится, есть $p_i\in\left(0;1\right]$. Игрок $S$ может смотреть в одну и ту же ячейку несколько раз, тратя на это попытки.

Цель игрока $H$ — спрятать объект в такую ячейку, чтобы вероятность обнаружения объекта игроком $S$ была минимальна.

Цель игрока $S$ — найти объект или максимально (с точки зрения вероятности обнаружения) приблизиться к этому.

Данная игра, как и любая матричная игра, может быть представлена в виде матрицы игры — таблицы, где строки отвечают за стратегии одного игрока ($S$ в данном случае), а столбцы — за стратегии другого игрока (соответственно, $H$).

На пересечении $i$-ой строки и $j$-ого столбца, в ячейке $(i, j)$ матрицы игры, находится выигрыш (проигрыш) игрока $S$ (игрока $H$) — вероятность того, что игрок $S$ найдёт объект в данной ячейке.

# Необходимые импорты и функции

Ниже находится необходимые для работы данного ноутбука технические детали — импорты, реализация необходимых функций (построение матрицы игры, сам метод Шепли-Сноу и пр.).

__Обратите внимание, что первую строчку в импорте при первом запуске нужно раскомментировать (удалить "# " в начале) — она нужна для установки необходимого для работы расширения. После установки строчку можно закомментировать обратно.__

In [ ]:
# !jupyter nbextension enable --py widgetsnbextension
import numpy as np
from math import comb, floor, ceil
from itertools import combinations, compress
import pandas as pd
from tqdm.notebook import tqdm, trange
import time
import matplotlib.pyplot as plt
from matplotlib.widgets import Slider, CheckButtons
%matplotlib notebook
from ipywidgets import *

In [ ]:
# global variables for all functions
prec = 10
eps = 1e-14

In [ ]:
def get_all_compositions(m, n, remainder = []):
    if n < 1:
        raise ValueError("n must be positive integer!")
    elif n == 1:
        return [remainder + [m]]
    else:
        answer = []
        for i in range(m, -1, -1):
            answer += get_all_compositions(m-i, n-1, remainder+[i])
    return answer

In [ ]:
def get_game_matrix(n, m, p):
    if not isinstance(n, int) or n < 1:
        raise ValueError(f"n must be positive integer, but was {type(n)}!")
    if not isinstance(m, int) or m < 1:
        raise ValueError(f"m must be positive integer, but was {type(m)}!")
    if len(p) != n:
        raise ValueError(f"Size of p must be equal to n, but was {len(p)}!")
    for i in range(n):
        if rounder(p[i], prec) < 0. or rounder(p[i] - 1., prec) > 0.:
            raise ValueError(f"Probability must be between 0 and 1 (ends are not included), but was {p[i]}")
    
    X = np.tile(np.array(p), (comb(m+n-1, m), 1))
    strategies = get_all_compositions(m, n)

    # contains following values: [p, p(1+q), p(1+q+q^2), ..., p(1+q+...+q^m)] 
    prob_list = [x * np.cumsum(np.cumprod(np.insert(np.repeat(1 - x, m - 1), 0, 1))) for x in p]
    for i in range(X.shape[0]):
        for j in range(X.shape[1]):
            X[i][j] = prob_list[j][strategies[i][j] - 1] if strategies[i][j] > 0 else 0
    return X, strategies

In [ ]:
def get_adjugate_matrix(matrix, det=None):
    if matrix.shape[0] != matrix.shape[1]:
        raise ValueError("Matrix should be square for computing its adjugate matrix!")
    
    if det is None:
        det = np.lonalg.det(matrix)
    if det != 0:
        C = np.linalg.inv(matrix) * det
    else :
        C = np.zeros(matrix.shape)
        nrows, ncols = C.shape
        for row in range(nrows):
            sign_factor = -1 if row % 2 == 1 else 1
            for col in range(ncols):
                minor = np.delete(np.delete(matrix, row, 0), col, 1)
                C[row][col] = sign_factor * np.linalg.det(minor)
                sign_factor = -sign_factor
    return C

In [ ]:
def group_equal(arr):
    groups = []
    for i in range(len(arr)):
        for j in range(i + 1, len(arr)):
            if arr[i] == arr[j]:
                if not groups:
                    groups.append([i, j])
                    continue
                found = False
                for g in groups:
                    if i in g:
                        if not j in g:
                            g.append(j)
                        found = True
                        break
                if not found:
                    groups.append([i, j])
    return groups

In [ ]:
def get_value(X, x=None, y=None, i=None, j=None):
    if i is not None:
        x = np.zeros(X.shape[0])
        x[i] = 1
    if j is not None:
        y = np.zeros(X.shape[1])
        y[j] = 1
    expectation = 0
    for k in range(len(x)):
        for l in range(len(y)):
            expectation += x[k] * y[l] * X[k][l]
    return expectation

In [ ]:
def choose_nearest_strategies(strategies, x, weak_condition=False):
    n = len(strategies[0])
    d = 2 if weak_condition else 1 
    lower_borders = np.array([max(floor(i) - d, 0) for i in x])
    upper_borders = np.array([min(ceil(i) + d, m) for i in x])
    def is_near_answer(strategy):
        s = np.array(strategy)
        return all(np.logical_and(lower_borders <= s, s <= upper_borders))
    near_strategies_mask = [is_near_answer(strategy) for strategy in strategies]
    return np.array(near_strategies_mask) if sum(near_strategies_mask) >= n else np.array([True]*len(strategies))
    # if sum(near_strategies_mask) >= n:
    #     X = X[near_strategies_mask]  
    #     strategies = list(compress(strategies, near_strategies_mask))
    # return X, strategies

In [ ]:
def choose_acceptable_strategies(probabilities, strategies, is_acceptable_strategy, diff_prob):
    n = len(strategies[0])
    acceptable_strategies_mask = [is_acceptable_strategy(probabilities, strategy, diff_prob) for strategy in strategies]
    if sum(acceptable_strategies_mask) < n:
        acceptable_strategies_mask = [is_acceptable_strategy(probabilities, strategy, diff_prob, True) for strategy in strategies]
        if sum(acceptable_strategies_mask) < n:
            acceptable_strategies_mask = [True]*len(strategies)
    return np.array(acceptable_strategies_mask)

In [ ]:
def is_acceptable_strategy(probabilities, strategy, diff_prob, week_condition=False):
    acceptable_strategy = True
    n = len(strategy)
    for i in range(n):
        for j in range(i, n):
            if diff_prob:
                if (rounder(probabilities[i] - probabilities[j], prec) < 0. and
                    strategy[i] - strategy[j] < -1 or
                    rounder(probabilities[i] - probabilities[j], prec) > 0. and
                    strategy[i] - strategy[j] > 1):
                    acceptable_strategy = False
                    break
            else:
                if (np.allclose(probabilities[i], probabilities[j], atol=eps) and
                    abs(strategy[i] - strategy[j]) > (1 if not week_condition else 2)):
                    acceptable_strategy = False
                    break
        if not acceptable_strategy:
            break
    return acceptable_strategy

In [ ]:
def find_equilibrium(_X, _strategies=None, probabilities=[], modded="10000", detailed_feedback=False, disable=False,
                    print_final_game_matrix=False):
    if not isinstance(modded, str) or len(modded) != 5:
        raise ValueError("Incorrect modded value!")
    for ch in modded:
        if ch != '0' and ch != '1':
            raise ValueError("Incorrect modded value!")
    start = time.time()
    total_counter = 0
    answer_x = []
    answer_y = []
    answer_value = None
    probabilities = np.array(probabilities)
    
    X = _X
    strategies = _strategies
    
    if modded[1] == '1' or modded[2] == '1' or modded[4] == '1':
        m = sum(_strategies[0])
        n = len(_strategies[0])
        acceptable_strategies_mask = np.array([True]*len(strategies))
        acceptable_strategies_eq_prob_mask = np.array([True]*len(strategies))
        near_strategies_mask = np.array([True]*len(strategies))
        if modded[1] == '1':
            acceptable_strategies_mask = choose_acceptable_strategies(probabilities, strategies,
                                                                      is_acceptable_strategy, True)
        if modded[2] == '1':
            acceptable_strategies_eq_prob_mask = choose_acceptable_strategies(probabilities, strategies,
                                                                             is_acceptable_strategy, False)
        if modded[4] == '1':
            _, x, _, _ = get_answer_by_input_data(n, 1, probabilities, detailed_feedback=detailed_feedback, modded="11110")
            x = x[0] * m
            weak_condition = np.any(rounder(probabilities - 0.8, prec) > 0.) and m >= 7
            near_strategies_mask = choose_nearest_strategies(strategies, x, weak_condition)
        final_strategies_mask = acceptable_strategies_mask * acceptable_strategies_eq_prob_mask * near_strategies_mask
        if sum(final_strategies_mask) < n:
            final_strategies_mask = acceptable_strategies_mask * acceptable_strategies_eq_prob_mask
        X = X[final_strategies_mask]    
        strategies = list(compress(strategies, final_strategies_mask))
    
    if print_final_game_matrix:
        print_game_matrix(X, strategies, title="Truncated matrix game")
    
    # ZERO_DET_COUNTER = 0
    # Choose sq_matrix_size rows from modded_X.shape[0] possible
    min_size = 1
    if modded[0] == '1':
        min_size = X.shape[1]
    for sq_matrix_size in trange(min_size, X.shape[1] + 1, disable=disable):
        for row_idxs in tqdm(combinations(range(X.shape[0]), sq_matrix_size),
                             total=comb(X.shape[0], sq_matrix_size), disable=disable):
            for col_idxs in combinations(range(X.shape[1]), sq_matrix_size):
                total_counter += 1

                try:
                    row_idxs = list(row_idxs)
                    col_idxs = list(col_idxs)
                    _row_idxs = [i for i in row_idxs]
                    _col_idxs = [i for i in col_idxs]
                    if _strategies is not None:
                        rows = [np.array(strategies[i]) for i in row_idxs]
                    
                    if modded[3] == '1':
                        is_acceptable_set = True
                        for i in range(len(rows)):
                            for j in range(i + 1, len(rows)):
                                diff = abs(rows[i] - rows[j])
                                if any(e > 1 for e in abs(rows[i] - rows[j])):
                                    is_acceptable_set = False
                                    break
                            if not is_acceptable_set:
                                break
                        if not is_acceptable_set:
                            continue
                    if modded[1] == '1' or modded[2] == '1' or modded[4] == '1':
                        j = 0
                        for i in range(len(_strategies)):
                            if _strategies[i] == strategies[row_idxs[j]]:
                                _row_idxs[j] = i
                                j += 1
                                if j == len(row_idxs):
                                    break
                    
                    B = X[np.ix_(row_idxs,col_idxs)]

                    if detailed_feedback:
                        print(f"Case #{total_counter}")
                        if _strategies is not None:
                            print(f"Square submatrix for rows {['{' + ', '.join(str(e) for e in x) + '}' for x in rows]}:")
                        # print(B)

                    J = np.ones((1, B.shape[1]))
                    detB = np.linalg.det(B)
                    # if np.allclose(detB, 0, atol=eps):
                    #     ZERO_DET_COUNTER += 1
                    adjB = get_adjugate_matrix(B, detB)
                    xnom = J.dot(adjB)
                    denom = xnom.dot(J.transpose())[0][0]
                    if np.allclose(denom, 0, atol=eps):
                        raise Exception("Denominator is zero!")
                    xnom = xnom[0]
                    ynom = J.dot(adjB.transpose())[0]
                    x = np.zeros(_X.shape[0])
                    x[np.ix_(_row_idxs)] = xnom / denom
                    y = np.zeros(_X.shape[1])
                    y[np.ix_(_col_idxs)] = ynom / denom
                    v = detB / denom
                    if np.any((rounder(x, prec) < 0.)) or np.any((rounder(y, prec) < 0.)):
                        raise Exception("Some of probabilities are negative! ")
                        
                    
                    for j in range(_X.shape[1]):
                        expectation = get_value(_X, x=x, j=j)
                        rounded_diff = rounder(expectation - v, prec)
                        rounded_y = rounder(y[j], prec)
                        if (rounded_diff > 0. and rounded_y > 0. or rounded_diff < 0.):
                            # print(f"rounded_diff={rounded_diff}; rounded_y={rounded_y}")
                            raise Exception(f"Check failed! E(x, {j + 1}) = {expectation} < v = {v}; eps={eps};")
                        elif detailed_feedback:
                            print(f"Check didn't fail! E(x, {j + 1}) = {expectation} >= v = {v}; eps={eps}")
                    for k in range(_X.shape[0]):
                        expectation = get_value(_X, y=y, i=k)
                        if (rounder(v - expectation, prec) > 0. and rounder(x[k], prec) > 0. or
                            rounder(expectation - v, prec) > 0. or
                            rounder(x[k], prec) > 0. and not np.allclose(expectation, v, atol=eps)):
                            if _strategies is not None:
                                raise Exception(f"Check failed! E({k + 1}, y) "
                                                f"= {expectation} > v = {v}; eps={eps}")
                            else:
                                raise Exception(f"Check failed! E({k + 1}, y) "
                                                f"= {expectation} > v = {v}; eps={eps}")
                        elif detailed_feedback:
                            if _strategies is not None:
                                print(f"Check didn't fail! E({'{' + ', '.join(str(e) for e in _strategies[k]) + '}'}, y) "
                                      f"= {expectation} <= v = {v}; eps={eps}")
                            else:
                                print(f"Check didn't fail! E({k + 1}, y) "
                                      f"= {expectation} <= v = {v}; eps={eps}")


                    if not answer_value:
                        answer_value = v
                    elif not np.allclose(answer_value, v, atol=eps):
                        raise RuntimeError("Game values are different for different pair of strategies!")

                    if detailed_feedback:
                        print("Got answer!")
                        print(f"x = {rounder(xnom/denom)}")
                        print(f"y = {rounder(y)}")
                        print()
                        print()

                    to_add = True
                    for _x in answer_x:
                        if np.allclose(x, _x):
                            to_add = False
                            break
                    if to_add:
                        answer_x.append(x)

                    to_add = True
                    for _y in answer_y:
                        if np.allclose(y, _y):
                            to_add = False
                            break
                    if to_add:
                        answer_y.append(y)

                except Exception as e:
                    if detailed_feedback:
                        print(e)
                        if not np.allclose(denom, 0, atol=eps):
                            print(f"x = {rounder(xnom/denom)}")
                            print(f"y = {rounder(y)}")
                        print()
                        print()
    # print(f"Zero determinants were found in {ZERO_DET_COUNTER} cases of {total_counter} total.")
    end = time.time()
    # if print_time:
    #     print(f"Execution time: {rounder(end - start)} seconds.")
    return answer_value, answer_x, answer_y, end-start

In [ ]:
def ordinal(n):
    return "%d%s" % (n,"tsnrhtdd"[(n//10%10!=1)*(n%10<4)*n%10::4])

In [ ]:
def check_equality(a, b):
    if len(a) != len(b):
        return False
    return all(np.allclose(x, y) for x, y in zip(a, b))

In [ ]:
def rounder(a, prec = 5):
    if isinstance(a, int) or isinstance(a, float) or isinstance(a, np.int32):
        return round(a, prec)
    elif isinstance(a, list) or isinstance(a, np.ndarray):
        return np.array([round(k, prec) for k in a])
    else:
        raise Exception(f"Unexpected argument for rounder function: {a} of type {type(a)}")

In [ ]:
def get_answer_by_input_data(n, m, p, modded="11111", detailed_feedback=False):
    X, strategies = get_game_matrix(n, m, p)
    return find_equilibrium(X, strategies, p, disable=True, modded=modded, detailed_feedback=detailed_feedback)[:3] + (strategies,)

In [ ]:
def print_game_matrix(X, strategies, detailed_labels=True, title="Matrix of game"):
    rows = ['{' + ', '.join(str(e) for e in x) + '}' for x in strategies]
    columns = [ordinal(x) for x in range(1, X.shape[1] + 1)]
    
    if detailed_labels:
        rows = [x + " times" for x in rows]
        columns = [y + " cells" for y in columns]
    
    df = pd.DataFrame(np.vectorize(rounder)(X), index=rows, columns=columns)
    print(title)
    display(df)
    print()

In [ ]:
def print_optimal_strategies(x, y, rows, columns):
    dfx = pd.DataFrame(np.array(rounder(x)).reshape(1, -1), columns=rows, index=['x'])
    dfy = pd.DataFrame(np.array(rounder(y)).reshape(1, -1), columns=columns, index=['y'])
    display(dfx.loc[:, [not (dfx[e] == 0).all() for e in dfx.columns]])
    display(dfy.loc[:, [not (dfy[e] == 0).all() for e in dfy.columns]])

In [ ]:
def print_result(v, x, y, strategies, n):
    if v is None:
        print(f"Couldn't found solution!")
        return
    print(f"Value of the game is {rounder(v)}.")
    print()
    
    print("Optimal strategies of player S (only non-zero values!):")
    columns = ['{' + ', '.join(str(e) for e in x) + '}' for x in strategies]
    counter = 0
    for s in x:
        counter += 1
        dfx = pd.DataFrame(np.array(rounder(s)).reshape(1, -1), columns=columns, index=[f'x{counter}'])
        display(dfx.loc[:, [not (dfx[e] == 0).all() for e in dfx.columns]])
    print()
    
    print("Optimal strategies of player H (only non-zero values!):")
    columns = [ordinal(x) for x in range(1, n + 1)]
    counter = 0
    for s in y:
        counter += 1
        dfy = pd.DataFrame(np.array(rounder(s)).reshape(1, -1), columns=columns, index=[f'y{counter}'])
        display(dfy.loc[:, [not (dfy[e] == 0).all() for e in dfy.columns]])

In [ ]:
def general_case_get_prob(p, k, m, b):
    if b == 0:
        return (1-p)**(m*k)
    s = 0
    for t in range(m):
        inner_sum = 0
        for u in range(0, b):
            inner_sum += (comb(k, u+1) * p**(u+1) * (1-p)**(k-u-1) *
                          general_case_get_prob(p, k-u-1, m-1-t, b-u-1))
        s += (1-p)**(t*k) * inner_sum
    return s

In [ ]:
def general_case_get_game_matrix(n, m, k, p):
    if not isinstance(n, int) or n < 1:
        raise ValueError(f"n must be positive integer, but was {type(n)}!")
    if not isinstance(m, int) or m < 1:
        raise ValueError(f"m must be positive integer, but was {type(m)}!")
    if not isinstance(k, int) or k < 1:
        raise ValueError(f"k must be positive integer, but was {type(k)}!")
    if len(p) != n:
        raise ValueError(f"Size of p must be equal to n, but was {len(p)}!")
    for i in range(n):
        if rounder(p[i], prec) < 0. or rounder(p[i] - 1., prec) > 0.:
            raise ValueError(f"Probability must be between 0 and 1 (ends are not included), but was {p[i]}")
    
    X = np.zeros((comb(m+n-1, m), comb(k+n-1, k)))
    strategies_s = get_all_compositions(m, n)
    strategies_h = get_all_compositions(k, n)
    for i in range(X.shape[0]):
        for j in range(X.shape[1]):
            s = 0
            for l in range(n):
                _k = strategies_h[j][l]
                _m = strategies_s[i][l]
                _p = p[l]
                values = np.array(list(range(_k+1)))
                probs = np.array([general_case_get_prob(_p, _k, _m, v) for v in values])
                s += np.dot(values, probs)
            X[i][j] = s
    return X, strategies_s, strategies_h

In [ ]:
def general_case_print_game_matrix(X, strategies_s, strategies_h):
    rows = ['{' + ', '.join(str(e) for e in x) + '}' for x in strategies_s]
    columns = ['{' + ', '.join(str(e) for e in x) + '}' for x in strategies_h]
    
    df = pd.DataFrame(np.vectorize(rounder)(X), index=rows, columns=columns)
    print("Matrix of game:")
    display(df)
    print()

In [ ]:
def general_case_print_result(v, x, y, strategies_s, strategies_h):
    if v is None:
        print(f"Couldn't found solution!")
        return
    print(f"Value of the game is {rounder(v)}.")
    print()
    
    print("Optimal strategies of player S (only non-zero values!):")
    columns = ['{' + ', '.join(str(e) for e in x) + '}' for x in strategies_s]
    counter = 0
    for s in x:
        counter += 1
        dfx = pd.DataFrame(np.array(rounder(s)).reshape(1, -1), columns=columns, index=[f'x{counter}'])
        display(dfx.loc[:, [not (dfx[e] == 0).all() for e in dfx.columns]])
    print()
    
    print("Optimal strategies of player H (only non-zero values!):")
    columns = ['{' + ', '.join(str(e) for e in x) + '}' for x in strategies_h]
    counter = 0
    for s in y:
        counter += 1
        dfy = pd.DataFrame(np.array(rounder(s)).reshape(1, -1), columns=columns, index=[f'y{counter}'])
        display(dfy.loc[:, [not (dfy[e] == 0).all() for e in dfy.columns]])

# Введите параметры и получите ответ

Параметры и ограничения:

$n$ — количество ячеек, $n \in \mathbb N$;

$m$ — количество попыток поиска у игрока $S$, $m \in \mathbb N$;

$p = (p_1, ..., p_n)$ — вероятности найти объект в соответствующих ячейках, если он там спрятан, $p_i \in (0; 1]$;

\_modded — используемые алгоритмом модификации;

\_detailed_feedback — переменная, которая нужна для дебага, но так-то она позволяет увидеть, что происходит внутри алгоритма — почему те или иные квадратные подматрицы были отклонены.

Модификации алгоритма:

1. Использовать квадратные подматрицы только максимальной размерности
2. Большей вероятности найти предмет в ячейке соответствует меньшее число попыток поиска, затрачиваемых на неё.
3. Равным вероятностям найти предмет в ячейке соответствуют близкие числа попыток поиска в них.
4. Оптимальные смешанные стратегии содержат с ненулевой вероятностью реализации только похожие друг на друга чистые стратегии.
5. Локализация решения с помощью решения подзадачи меньшей размерности.

Подробное описание  модификаций находится в ВКР по ссылке: [здесь будет ссылка, когда диплом будет выложен].

Модификации задаются ключом — строка, размерности 5, символы которой принимают значение либо "1", либо "0" — модификация включена или выключена, соответственно.

Под урезанной матрицей игры в выводе подразумевается такая подматрица исходной, только квадратные подматрицы которой будут рассматриваться для поиска решений. Для проверки подозрительных на оптимальность стратегий при этом используется оригинальная неурезанная матрица.

Отметим, что если алгоритм вывел несколько оптимальных стратегий для какого-то из игроков, то их следует воспринимать как крайние точки выпуклого многогранника, все значения которого являются оптимальными стратегиями данного игрока.

In [ ]:
# Enter parameters here
n = 2
m = 4
p = np.array([0.5, 0.3])
_modded = "11110"
_detailed_feedback = False

try:
    X, strategies = get_game_matrix(n, m, p)
    print_game_matrix(X, strategies)
    value, x, y, t = find_equilibrium(X, strategies, p, modded=_modded, detailed_feedback=_detailed_feedback,
                                  print_final_game_matrix=True)
    print(f"Time: {t} sec.")
    if value is None:
        print(f"n={n}; m={m}; p={p}; TYPE ERROR")
        print(f"v1={value}; x1={x}; y1={y}")
    print_result(value, x, y, strategies, X.shape[1])
except Exception as e:
    raise e
    print(f"Error! {e}")

# Тестирование модификаций алгоритма — совпадает ли ответ с немодифицированной версией:

Какие модификации использовать, а какие нет — регулируется переменной _modifications_key_. описание модификаций: см. пред. часть.

Выводится будут только параметры задач, которые показали разный ответ на модифицированной и немодифицированной версии, и собственно, ответы на них. Однако это всё ещё требует ручной проверки, посколько в некоторых случаях выводятся одинаковые ответы.

Если включена пятая модификация, то иногда могут выводиться также "Type Error". Эта ошибка выводится в том случае, если значение игры оказалось None вместо числа. Такое происходит, если ни одна квадратная матрица, подозрительная на оптимальность, не прошла проверку на оптимальность. Такого в обычной игре быть не может, но поскольку пятая модификация урезает количество рассматриваемых квадратных подматриц, то "правильная" квадратная подматрица вполне могла не войти во множество рассматриваемых, а значит ответ вполне мог быть не найден. Данная ошибка возникает только при больших $m$ и $n$ (>=7).

In [ ]:
nmin = 1
nmax = 3
mmin = 1
mmax = 12
pstep = 0.1
pmin = 0.1
pmax = 1 - pstep
pstep_int = int((pmax - pmin) / pstep) + 1

modifications_key="11110"

for n in trange(nmin, nmax + 1):
    p = np.full(n, pmin)
    for _ in trange(pstep_int ** n):
        for m in range(mmin, mmax + 1):
            # print(f"\n\n{p}\n\n")
            X, strategies = get_game_matrix(n, m, p)
            v1, x1, y1, _ = find_equilibrium(X, strategies, p, modded=modifications_key, disable=True)
            v2, x2, y2, _ = find_equilibrium(X, strategies, p, modded="00000", disable=True)
            try:
                if v1 is None:
                    print(f"n = {n}, p = {rounder(p)}, m = {m}; v1 is None")
                    continue
                if v2 is None:
                    print(f"n = {n}, p = {rounder(p)}, m = {m}; v2 is None")
                    continue
                if v1 != v2 or not check_equality(x1, x2) or not check_equality(y1, y2):
                    print(f"n = {n}, p = {rounder(p)}, m = {m}\nModded results:")
                    print_result(v1, x1, y1, strategies, n)
                    print("\nUnmodded results:")
                    print_result(v2, x2, y2, strategies, n)
                    print("\n===================\n\n")
            except TypeError as e:
                # pass
                print(f"n={n}; m={m}; p={p}; TYPE ERROR")
                print(f"v1={v1}; x1={x1}; y1={y1}")
        p[n - 1] += pstep
        for i in range(1, n+1):
            if np.allclose(p[n - i], pmax+pstep):
                p[n - i] = pmin
                if i != n:
                    p[n - 1 - i] += pstep

# Интерактивный график

Единственный параметр здесь, который предполагается, что можно менять, в коде ноутбука — $m$, количество попыток поиска. Всё остальное предполагается, что должно остаться нетронутым. На интерактивном графике, разумеется, можно менять все ползунки и чек-боксы какие угодно и куда захочется.

In [ ]:
pstep = 0.01
pmin = pstep
pmax = 1. - pstep
pn = int(1./pstep) - 1

n = 2
m = 3

answers_v = np.zeros((pn, pn))
answers_x = np.zeros((m + 1, pn, pn))
answers_y_min = np.zeros((n, pn, pn))
answers_y_max = np.zeros((n, pn, pn))

p1 = pmin
for i in trange(pn):
    p2 = pmin
    for j in range(pn):
        v, x, y, _ = get_answer_by_input_data(n, m, [p1, p2])
        answers_v[i][j] = v
        for k in range(m + 1):
            try:
                answers_x[k][i][j] = x[0][k]
            except IndexError as e:
                print(f"Error! i = {i}, j = {j}, k = {k}, p = {[p1, p2]}, x = {x}, y = {y}")
        for k in range(n):
            if len(y) == n:
                answers_y_min[k][i][j] = min(y[0][k], y[1][k])
                answers_y_max[k][i][j] = max(y[0][k], y[1][k])
            else:
                answers_y_min[k][i][j] = y[0][k]
                answers_y_max[k][i][j] = y[0][k]
        p2 += pstep
    p1 += pstep

In [ ]:
p1 = np.linspace(pmin, pmax, pn)

# Define initial parameters
init_p2 = 0.5

# Create the figure and the line that we will manipulate
fig, ax = plt.subplots()
idx = int((pn+1)*init_p2-1)
v_line = plt.plot(p1, answers_v[:, idx], lw=2, label="v")
x_lines = [plt.plot(p1, answers_x[i, :, idx], lw=2, label=f"x{i+1}") for i in range(m + 1)]
y_lines = [plt.plot(p1, answers_y_min[i, :, idx], lw=2, label=f"y{i+1}") for i in range(n)]

idxs = [i for i in range(n)]
for i in range(n):
#     idxs.insert(n - i, 2 * n - 1 - i)
    idxs.insert(n - i, n - 1 - i)
print(idxs)
# lines = [v_line] + x_lines + [(y_min_lines + y_max_lines)[i] for i in idxs]
lines = [v_line] + x_lines + y_lines
ax.set_xlabel('p1')

# adjust the main plot to make room for the sliders
plt.subplots_adjust(right=0.8, bottom=0.25)
plt.ylim([0, 1])
plt.title(f"n = {n}, m = {m}")
fig.set_size_inches(10, 8)

# Make a horizontal slider to control the frequency.
axp2 = plt.axes([0.1, 0.1, 0.8, 0.03])
p2_slider = Slider(
    ax=axp2,
    label='p2',
    valmin=pmin,
    valmax=pmax,
    valinit=init_p2,
    valstep=pstep,
)

# The function to be called anytime a slider's value changes
def update(val):
    idx = int((pn+1)*p2_slider.val-1)
    lines[0][0].set_ydata(answers_v[:, idx])
    for i in range(1, m + 2):
        lines[i][0].set_ydata(answers_x[i-1, :, idx])
    # for i in range(m + 2, m + 2 + 2 * n, 2):
    #     lines[i][0].set_ydata(answers_y_min[int((i-m-2)/2), :, idx])
    #     lines[i + 1][0].set_ydata(answers_y_max[int((i-m-2)/2), :, idx])
    for i in range(m + 2, m + 2 + n):
        lines[i][0].set_ydata(answers_y_min[int(i-m-2), :, idx])
    fig.canvas.draw_idle()

# register the update function with each slider
p2_slider.on_changed(update)

rax = plt.axes([0.81, 0.25, 0.123, 0.32])
labels = [str(line[0].get_label()) for line in lines]
visibility = [line[0].get_visible() for line in lines]
check = CheckButtons(rax, labels, visibility)

def func(label):
    index = labels.index(label)
    lines[index][0].set_visible(not lines[index][0].get_visible())
    plt.draw()

check.on_clicked(func)

ax.legend(loc=(1.015, 0.53), prop={'size': 12})

plt.show()


# Вывод матрицы игры для обобщения задачи

Разница здесь в том, что добавляется новый параметр — $k$, количество предметов, которые прячет игрок $H$.
Заметим, что параметр $_modded$ здесь лучше тоже не трогать, поскольку модификации, описанные для частного случая выше, гарантированно работают только для него.

In [ ]:
# Enter parameters here
n = 2
m = 8
k = 1
p = np.array([0.5, 1/3])
_detailed_feedback = False

try:
    X, strategies_s, strategies_h = general_case_get_game_matrix(n, m, k, p)
    general_case_print_game_matrix(X, strategies_s, strategies_h)
    
    value, x, y, _ = find_equilibrium(X, modded="00000", detailed_feedback=_detailed_feedback)
    
    if value is None:
        print(f"n={n}; m={m}; p={p}; TYPE ERROR")
        print(f"v1={value}; x1={x}; y1={y}")
    general_case_print_result(value, x, y, strategies_s, strategies_h)
except Exception as e:
    raise e
    print(f"Error! {e}")

# Подпрограмма для прогона тестов на различных модификациях и задачах

Использовалась для написания главы 6 ВКР

In [ ]:
#Time execution testings

try:
    keys = ["00000", "00001", "00010", "00011", "00100", "00101", "00110", "00111",
            "01000", "01001", "01010", "01011", "01100", "01101", "01110", "01111",
            "10000", "10001", "10010", "10011", "10100", "10101", "10110", "10111",
            "11000", "11001", "11010", "11011", "11100", "11101", "11110", "11111"]
    tasks = [(4, 6, [0.5, 0.2, 0.6, 0.8]), (4, 3, [0.5, 0.2, 0.6, 0.8]),
             (3, 6, [0.5, 0.2, 0.6]), (4, 6, [0.5, 0.5, 0.5, 0.5]),
             (4, 6, [0.5, 0.5, 0.6, 0.8]), (4, 6, [0.5, 0.5, 0.8, 0.8])]
    solo_keys = ["00100", "01000", "00010", "00001", "10000"]
    keys.reverse()
    for n, m, p in tasks:
        if p[1] != 0.5:
            continue
        print(f"Current task: n = {n}; m = {m}; p = {p}")
        if n == 4 and m == 6 and p[len(p) - 1] == 0.5:
            _keys = ["00000", "01000"]
        else:
            _keys = keys
        for key in _keys:
            print(f"key: {','.join([str(i+1) for i in range(len(key)) if key[i] == '1'])}")
            X, strategies = get_game_matrix(n, m, p)
            value, _, _, exec_time = find_equilibrium(X, strategies, p, modded=key, disable=True)
            print(f"Time: {exec_time}")
            if value is None:
                print(f"TYPE ERROR was caught!")
            print("\n")
        print("\n\n\n\n")
except Exception as e:
    raise e
    print(f"Error! {e}")